# How to build simple Agent LLM App with LangGraph

## Concepts included
* LangGraph

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [4]:
!pip list | grep langchain

langchain                 0.2.1
langchain-community       0.2.1
langchain-core            0.2.3
langchain-openai          0.1.8
langchain-text-splitters  0.2.0

[notice] A new release of pip is available: 23.1.2 -> 24.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


## Connect with LLM and start conversation with it

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

### Track operation in LangSmith
* [Open LangSmith here](https://smith.langchain.com/)

## Agents
* Agents use LLMs as reasoning engines to determine which actions to take

## Tool-using Agent
* We will need to add our Tavily API key in .env file
* TAVILY_API_KEY=

In [7]:
#pip install langgraph

### Define tools

In [24]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("Who are the top stars of the 2024 Eurocup?")

[{'url': 'https://www.nbcsports.com/soccer/news/euro-2024-player-power-rankings-who-are-the-top-20-stars',
  'content': '4. Kevin de Bruyne (Belgium) 3. Rodri (Spain) 2. Harry Kane (England) 1. Kylian Mbappe (France) On the bubble: Andriy Yarmolenko, Alvaro Morata, Dani Olmo, Benjamin Sesko, Aleksandar Mitrovic, Bernardo Silva, Micky van de Ven, Jeremie Frimpong, Davide Frattesi, Leroy Sane, Manuel Neuer, Florian Wirtz, Antonio Rudiger, Khvicha Kvaratskhelia ...'},
 {'url': 'https://www.theringer.com/soccer/2024/6/12/24176241/euro-2024-power-rankings',
  'content': '16. Ukraine (Group E) Ukraine has more talent than its underlying statistical profile would suggest it does. It is one of three teams to qualify with a negative xG differential in qualifying ...'}]

In [10]:
tools = [search]

In [11]:
# In order to enable this model to do tool calling we use .bind_tools to give language model knowledge of these tools

llm_with_tools = llm.bind_tools(tools)

## Create agent

In [12]:
from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

## Run agent

In [19]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', id='25bbe305-75d5-4442-ae81-94fcf4dda6bf'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VLtoqg91gfUSTOai9oro2VeW', 'function': {'arguments': '{"query":"Soccer Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 92, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0ce3ef27-88f5-439c-b1e2-54179ebb6789-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Soccer Eurocup 2024 location'}, 'id': 'call_VLtoqg91gfUSTOai9oro2VeW'}], usage_metadata={'input_tokens': 92, 'output_tokens': 25, 'total_tokens': 117}),
 ToolMessage(content='[{"url": "https://www.uefa.com/euro2024/news/0273-14ac084902cc-9ef59ec61319-1000--euro-2024-host-cities-venue-guide/", "content": "BVB Stadion Dortmund\\nB

In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LD97FzexV807GdQVgeqcl3IF', 'function': {'arguments': '{"query":"2024 Eurocup final match date and location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 96, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-633295c6-e95c-4cc9-95ff-eadc64c36098-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match date and location'}, 'id': 'call_LD97FzexV807GdQVgeqcl3IF'}], usage_metadata={'input_tokens': 96, 'output_tokens': 26, 'total_tokens': 122})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.uefa.com/euro2024/news/0284-18bb952a9458-2a9e1ff202c4-1000--euro-2024-final-when-and-where-is-it/", "content": "The UEFA EURO 2024 final will take place on Sunday